In [31]:
import torch

In [ ]:
class LDPTwoSampleTester:
    def __init__(self)

        

In [59]:
def TransformIntervalIndex(data, nIntervals):
  # for each dimension, transform the data in [0,1] into the interval index
  # first interval = [0, x], the others = (y z]
    
    # data: torch tensor object
    # n.intervals: integer
  
  # create designated number of intervals
    if data.dim() == 1:
        d = 1
    elif data.dim() == 2:
        d = data.size(dim = 1)
    else:
        return # we only use up to 2-dimensional tensor

    breaks = torch.linspace(start = 0, end = 1, steps = nIntervals + 1)
    dataIndices = torch.bucketize(data, breaks, right = False) # ( ] form.
    dataIndices = dataIndices + (dataIndices==0) #move 0 values from the bin number 0 to the bin number 1
    return(dataIndices)


In [253]:
data = torch.tensor(
[
[0.19901582, 0.29330425, 0.08031318, 0.27744206],
[0.38371595, 0.07725842, 0.58872328, 0.60947456],
[0.78765378, 0.18596928, 0.20049580, 0.04321161],
[0.60499579, 0.38050702, 0.26301983, 0.58410214],
[0.42093993, 0.42060113, 0.89575178, 0.57233768],
[0.01160462, 0.55440856, 0.50919182, 0.80756614]
]
)



In [252]:
data.fill_diagonal_(0)

tensor([[0.0000, 0.2933, 0.0803, 0.2774],
        [0.3837, 0.0000, 0.5887, 0.6095],
        [0.7877, 0.1860, 0.0000, 0.0432],
        [0.6050, 0.3805, 0.2630, 0.0000],
        [0.4209, 0.4206, 0.8958, 0.5723],
        [0.0116, 0.5544, 0.5092, 0.8076]])

# 1. test of TransformIntervalIndex

In [88]:
index_1 = TransformIntervalIndex(data, 4)
index_1

tensor([[1, 2, 1, 2],
        [2, 1, 3, 3],
        [4, 1, 1, 1],
        [3, 2, 2, 3],
        [2, 2, 4, 3],
        [1, 3, 3, 4]])

In [84]:
index_2 = TransformIntervalIndex(data, 5)
index_2

tensor([[1, 2, 1, 2],
        [2, 1, 3, 4],
        [4, 1, 2, 1],
        [4, 2, 2, 3],
        [3, 3, 5, 3],
        [1, 3, 3, 5]])

In [85]:
index_3 = TransformIntervalIndex(data, 6)
index_3

tensor([[2, 2, 1, 2],
        [3, 1, 4, 4],
        [5, 2, 2, 1],
        [4, 3, 2, 4],
        [3, 3, 6, 4],
        [1, 4, 4, 5]])

In [113]:
index_3.type()

'torch.LongTensor'

In [122]:
def TransformMultivariate(dataInterval, nBin):
    dim = dataInterval.size(dim = 1)   
    return(
        1 + torch.matmul(
            (dataInterval - 1),
            torch.pow(
                self = nBin,
                exponent = torch.linspace(start = (dim-1), end = 0, steps = dim, dtype = torch.long)
            )
        )
    ) 

# 2.1 Test of TransformMultivariate

In [125]:
multi_1 = TransformIntervalIndex(index_1, 4);
multi_1

tensor([ 18,  75, 193, 151,  95,  44])

In [126]:
multi_2 = TransformMultivariate(index_2, 5)
multi_2

tensor([ 27, 139, 381, 408, 323,  65])

In [128]:
multi_3 = TransformMultivariate(index_3, 6)
multi_3
multi_3.type()

'torch.LongTensor'

# 3. implementation of TransformOnehot

In [137]:
def TransformOnehot(dataMultivariate, newdim):
    return(
        torch.nn.functional.one_hot(
            dataMultivariate-1,
            num_classes = newdim)
    )


In [218]:
onehot_1 = TransformOnehot(multi_1, 4**4)
onehot_1

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [221]:
torch.arange(256).reshape(2,-1)

tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
          56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
          84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
          98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
         126, 127],
        [128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
         142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
         156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
         170, 171, 172, 173, 174

# implementation of noise.conti

In [183]:
def noise_conti(n, dim, alpha):
      #dim = kappa^d for conti data, d for discrete data
    unit_laplace_generator = torch.distributions.laplace.Laplace(torch.tensor([0.0]), torch.tensor([2**(-1/2)]))
    laplace_samples = unit_laplace_generator.sample(sample_shape = torch.Size([n * dim]))
    scale = (8**(1/2) / alpha) * (dim**(1/2))
    return(scale*laplace_samples.reshape(-1))

In [200]:
torch.var(noise_conti(1000, 2, 0.5))


tensor(63.8272)

# implementation of noise.discrete

In [208]:
def noise_discrete(n, dim, alpha):
    #dim = kappa^d for conti data, d for discrete data
    param_geom = 1 - torch.exp(torch.tensor(-alpha / (2* (dim**(1/2)) )))
    n_noise =  n * dim
    geometric_generator = torch.distributions.geometric.Geometric(param_geom)
    noise = geometric_generator.sample((n_noise,)) - geometric_generator.sample((n_noise,))
    return(noise.reshape(-1))


In [212]:
torch.var(noise_discrete(1000, 2, 0.5))



tensor(59.2451)

# implementation of PrivatizeTwoSample

In [242]:
def privatize_twosample(data, alpha = float("inf"), discrete_noise = False):
    ## assume the data is discrete by nature or has already been dicretized.
    n = data.size(dim = 0)
    dim = data.size(dim = 1) #kappa^d if conti data, d if discrete data
    scale = torch.tensor(dim**(1/2))
      
    if alpha == float("inf"): #non-private case
        return( torch.mul(scale, data) )
    else: # private case
        if discrete_noise:
          noise = noise_discrete(n = n, dim = dim, alpha = alpha)
        else:
          noise = noise_conti(n = n, dim = dim, alpha = alpha)
    print(noise)
    return(
        torch.add(
            input = data,
            alpha = scale,
            other = noise.reshape(n, -1)
        )
    )


In [243]:
privatize_twosample(data, alpha = 0.5)

tensor([ 17.9675,  -6.2361,  15.2690,  -3.3067,  19.4458,   1.1035,   3.8195,
          0.6491,  18.5508,   7.9279,   5.2640,   0.2620,   7.4066,   6.9240,
          6.9025,  -7.4443,   1.0893,   5.2800,  -2.4739, -15.4348,  -0.8586,
         20.9370,   3.0041,   6.2779])


tensor([[ 36.1340, -12.1788,  30.6183,  -6.3360],
        [ 39.2753,   2.2843,   8.2278,   1.9077],
        [ 37.8892,  16.0418,  10.7284,   0.5672],
        [ 15.4181,  14.2285,  14.0679, -14.3044],
        [  2.5996,  10.9805,  -4.0520, -30.2972],
        [ -1.7057,  42.4284,   6.5174,  13.3634]])

# u-statisitc function

In [260]:
def U_stat_twosample(data, n_1):
  n_2 = data.size(dim = 0) - n_1
  
  data_x = data[ :n_1, ]
  data_y = data[n_1: , ]
    
  # x only part
  u_x = torch.matmul(data_x, torch.transpose(data_x, 0, 1))
  u_x.fill_diagonal_(0)
  u_x = torch.sum(u_x) / (n_1 * (n_1 - 1))
  
  # y only part
  u_y = torch.matmul(data_y, torch.transpose(data_y, 0, 1))
  u_y.fill_diagonal_(0)
  u_y = torch.sum(u_y) / (n_2 * (n_2 - 1))
  
  # x, y part
  u_xy = torch.matmul(data_x, torch.transpose(data_y, 0, 1))
  u_xy = torch.sum(u_xy) * ( 2 / (n_1 * n_2) )
  
  return(u_x + u_y - u_xy)

In [259]:
U_stat_twosample(data, 2)

tensor(-0.1087)

In [237]:



      




noise.vairance.theoretic.conti <-function(dim, alpha){
  return(8 * dim / (alpha^2))
}

noise.vairance.theoretic.discrete <-function(dim, alpha){
  p <- exp(-alpha/(2*sqrt(dim)))
  return(2*p/(1-p)^2)
}




PrivatePermutationTwoSampleTest <-
  function(B, data.x, data.y, kappa, alpha, gamma, discrete = FALSE) {
    n.1 <- nrow(data.x)
    n.2 <- nrow(data.y)
    d <- ncol(data.x)
    
    data.x.binned <- Bin(data.x, kappa)
    data.y.binned <- Bin(data.y, kappa)
    
    data.combined <- rbind(data.x.binned, data.y.binned)
    data.privatized <- PrivatizeTwoSample(data.combined, alpha)
    ustat.original <- UstatTwoSample(data.privatized, n.1)
    #cat("\nunpermuted statistic:", ustat.original)
    #permutation procedure
    perm.stats <- rep(0, B)
    for (rep in 1:B) {
      perm <- sample(1:(n.1 + n.2)) 
      perm.stats[rep] <- UstatTwoSample(data.privatized[perm, ], n.1)
      #cat("\ntest statistic:", UstatTwoSample(data.privatized[perm, ], n.1))
    }
    p.value.proxy <- (1 + sum(ustat.original < perm.stats)) / (B + 1)
    
    #test result: TRUE = 1 = reject the null, FALSE = 0 = retain the null.
    
    return(p.value.proxy < gamma)
  }

L2distBetaUnif <- function(shape.1, shape.2){
  return(Beta(2 * shape.1 - 1, 2 * shape.2 -1) / (Beta(shape.1, shape.2))^2 - 1)
}

L2distBetaBeta <- function(shape.1.1, shape.1.2, shape.2.1, shape.2.2){
  first.beta.term <- Beta(2 * shape.1.1 - 1, 2 * shape.1.2 -1) / (Beta(shape.1.1, shape.1.2))^2
  second.beta.term <- Beta(2 * shape.2.1 - 1, 2 * shape.2.2 -1) / (Beta(shape.2.1, shape.2.2))^2
  cross.term <- Beta(shape.1.1 + shape.2.1 - 1, shape.1.2 + shape.2.2 -1) / ( (Beta(shape.1.1, shape.1.2)) * (Beta(shape.2.1, shape.2.2)) )
  return(first.beta.term + second.beta.term - 2 * cross.term)
}

Beta <- function(shape.1, shape.2){
  return(gamma(shape.1) * gamma(shape.2) / gamma(shape.1 + shape.2))
}

SyntaxError: invalid syntax (2715841981.py, line 1)